In [97]:
import os
from collections import Counter
import numpy as np
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.svm import SVC, NuSVC, LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [77]:
def make_Dictionary(train_dir):
    emails = [os.path.join(train_dir,f) for f in os.listdir(train_dir)] 
    all_words = [] 
    for mail in emails: 
        with open(mail, encoding="utf8", errors='ignore') as m:
            for i,line in enumerate(m):
                if i == 2: #Body of email is only 3rd line of text file
                    words = line.split()
                    all_words += words
                    dictionary = Counter(all_words) 
                    
    list_to_remove = list(dictionary.keys())
    for item in list_to_remove:
        if item.isalpha() == False: 
            del dictionary[item]
        elif len(item) == 1:
            del dictionary[item]

    dictionary = dictionary.most_common(3000)
    
    return dictionary

In [78]:
def extract_features(mail_dir,dictionary): 
    files = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)]
    features_matrix = np.zeros((len(files),3000))
    docID = 0;
    for fil in files:
        with open(fil, encoding="utf8", errors='ignore') as fi:
            for i,line in enumerate(fi):
                 if i == 2:
                        words = line.split()
                        for word in words:
                            wordID = 0
                            for i,d in enumerate(dictionary):
                                if d[0] == word:
                                    wordID = i
                                    features_matrix[docID,wordID] = words.count(word)
        docID = docID + 1

    return features_matrix

In [79]:
ham_filePath = '/Users/liulijia/Downloads/enron1/ham'
spam_filePath = '/Users/liulijia/Downloads/enron1/spam'

In [80]:
ham_len=len(os.listdir(ham_filePath))
spam_len=len(os.listdir(spam_filePath))

In [81]:
dict_ham=make_Dictionary(ham_filePath)
ham_matrix=extract_features(ham_filePath,dict_ham)

dict_spam=make_Dictionary(spam_filePath)
spam_matrix=extract_features(spam_filePath,dict_spam)

In [90]:
x=np.concatenate((ham_matrix,spam_matrix),axis=0)
y= np.zeros(ham_len+spam_len)
y[ham_len:]=1

In [92]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5)

In [93]:
model1 = MultinomialNB()
model2 = LinearSVC()

In [98]:
model1.fit(x_train,y_train)
result1 = model1.predict(x_test)
print(confusion_matrix(y_test,result1))

[[1668  184]
 [ 654   80]]


In [99]:
model2.fit(x_train,y_train)
result2 = model2.predict(x_test)
print(confusion_matrix(y_test,result2))

[[1455  397]
 [ 550  184]]
